### Using vessel time exposure (hours/km^2) to determine vessel type

In [1]:
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
import arrow
from datetime import datetime 
from datetime import datetime, date, timedelta
from numpy.random import choice

from get_date import get_date
from get_lat_lon_indices import get_lat_lon_indices

#### User inputs of lat, lon and month

In [2]:
start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(hours = 1)
ais_data_year   = 2018
n_locations     = 1  #desired number of latitude and longitude values
upsample_factor = 1
geotiff_directory = "/Users/rmueller/Documents/UBC/MIDOSS/Data/AIS/ShipTrackDensityGeoTIFFs/"
vessel_types      = ["tanker", "atb", "barge", "cargo", "cruise",
                     "ferry", "fishing", "smallpass", "other"]

#### Get all traffic data


In [3]:
spill_date = get_date(start_date, end_date, delta_time)
spill_month = spill_date.month  # numeric month
spill_month_name  = arrow.get(spill_date).format('MMMM') # string

In [4]:
all_traffic_filepath = geotiff_directory + f'all_{ais_data_year}_{spill_month:02.0f}.tif'
all_traffic_filepath

'/Users/rmueller/Documents/UBC/MIDOSS/Data/AIS/ShipTrackDensityGeoTIFFs/all_2018_12.tif'

####  Get spill lat/lon value(s) as well as index values for extracting vessel time exposure values from GeoTIFFS

In [5]:
[lats, lons, x_index, y_index, data_val] = get_lat_lon_indices(geotiff_directory, spill_month, n_locations, upsample_factor)    

Randomly selecting spill location from all-traffic GeoTIFF:
...Creating 2D array of lat/lon strings (this may take a moment)
...Selecting 1 location(s)
...Creating vectors of lat/lon pairs and x-index/y-index pairs
That's a wrap, folks!


#### load raster images
All geotiffs are named as: "vesseltype_yyyy_mm.tif"

In [43]:
vessel_type = []

for location in range(n_locations):
    
    # initialize array used to store vessel time exposure values for each vessel type
    VTE_by_vessel_and_location = []
    
    # loop through each vessel type and store VTE for each vessel type, at selected location
    for name in vessel_types:

        traffic_reader = rio.open(f'{geotiff_directory}{name}_{ais_data_year}_{spill_month:02.0f}.tif')

        # dataset closes automatically using the method below
        with traffic_reader as dataset:

            # resample data to target shape
            data = dataset.read(1,
                out_shape=(
                    dataset.count,
                    int(dataset.height),
                    int(dataset.width)
                ),
            )

            # Set all no-data values to zero
            data[data < 0] = 0
        
        # Store vessel time exposure [hours/km^2] for each vessel-type in array
        VTE_by_vessel_and_location.append(data[x_index[location],y_index[location]])
    
    # Calculate relative probability of vessel occurance based on VTE by vessel-type
    probability = VTE_by_vessel_and_location/np.sum(VTE_by_vessel_and_location)
    
    # Randomly select vessel type based on relative vessel time exposure probability
    vessel_random = choice(vessel_types, 
                          n_locations , 
                          p = probability)
    
    # populate vessel type selection for each n-locations
    vessel_type.append(vessel_random)